In [ ]:
import os, sys, re
import numpy as np
import pandas as pd
import matplotlib
import pickle
import tarfile
from datetime import datetime
from subprocess import call, Popen
from scipy import interp
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt

In [ ]:
!pip install feature_engine
!pip install unidecode
!pip install category_encoders

In [ ]:
matplotlib.use("pdf")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive/kalapa

/content/drive/My Drive/kalapa


In [ ]:
train_df = pd.read_csv("/content/drive/My Drive/kalapa/dataset/train.csv")
test_df = pd.read_csv("/content/drive/My Drive/kalapa/dataset/test.csv")
df_all = train_df.drop(['label'], 1).append(test_df)

print("Train: ", train_df.shape, " | Test: ", test_df.shape, " |  All: ", df_all.shape)

print("-"*70)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Train:  (53030, 195)  | Test:  (20381, 194)  |  All:  (73411, 194)
----------------------------------------------------------------------


In [ ]:
##Updated start here
DROP = (["gioiTinh","info_social_sex", 'currentLocationLocationId', 'currentLocationLatitude', 'currentLocationLongitude', 'homeTownLocationId', 'homeTownLatitude', 'homeTownLongitude', 'currentLocationName', 'homeTownName', 'diaChi'] + 
[f"Field_{c}" for c in [1, 3, 4, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 47, 50, 51, 52, 53, 54, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 76, 77]] + 
[  'partner0_A', 'partner0_B', 'partner0_C', 'partner0_D', 'partner0_E', 'partner0_F', 'partner0_H', 'partner0_H', 'partner0_K', 'partner0_L', 
   'partner1_B', 'partner1_C', 'partner1_D', 'partner1_E', 'partner1_F', 'partner1_G', 'partner1_H', 'partner1_K', 'partner1_L', 
   'partner2_A', 'partner2_B', 'partner2_C', 'partner2_D', 'partner2_E', 'partner2_G', 'partner2_H', 'partner2_K', 'partner2_L', 
   'partner3_B', 'partner3_C', 'partner3_E', 'partner3_F', 'partner3_G', 'partner3_H', 'partner3_K', 'partner3_L', 
   'partner4_A', 'partner4_B', 'partner4_C', 'partner4_D', 'partner4_E', 'partner4_F', 'partner4_G', 'partner4_H', 'partner4_K', 'partner4_L', 
   'partner5_B', 'partner5_C', 'partner5_D', 'partner5_E', 'partner5_F', 'partner5_G','partner5_H', 'partner5_K', 'partner5_L'])

cat_features_count_encode = ['Field_4', 'Field_12', 'Field_18', 'Field_34', 'gioiTinh', 'diaChi', 'Field_36',
                             'Field_38', 'Field_45', 'Field_46', 'Field_47', 'Field_48', 'Field_49',
                             'Field_54', 'Field_55', 'Field_56', 'Field_61', 'Field_62', 'Field_65', 'Field_66',
                             'Field_68', 'maCv', 'info_social_sex', 'data.basic_info.locale',
                             'currentLocationCountry', 'currentLocationState', 'homeTownCountry', 'homeTownState', 'brief']

unidecode_columns = 'diaChi currentLocationCountry currentLocationState homeTownCountry homeTownState maCv'.split()
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]
##End

In [ ]:
def subtract_date(date1,date2, df):
    df[date1] = pd.to_datetime(df[date1], infer_datetime_format=True)
    df[date2] = pd.to_datetime(df[date2], infer_datetime_format=True)
    df[date1+date2] = (df[date2] - df[date1]).dt.days
    
def process_ngaySinh(s):
    if s != s:
        return np.nan
    try:
        s = int(s)
    except ValueError:
        s = s.split(" ")[0]
        
    return datetime.strptime(str(s)[:6], "%Y%m")

def datetime_normalize(s):
    if s != s:
        return np.nan
        
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj

def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj
  
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        if col == "Field_34":
            continue
        df[col] = df[col].apply(date_normalize)

    df["Field_34"] = df["Field_34"].apply(process_ngaySinh)
    df["ngaySinh"] = df["ngaySinh"].apply(process_ngaySinh)
    
    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        #df[col].fillna(0)
        df[col] = df[col].dt.strftime('%m-%Y') #%d-%m-%Y
    
    subtract_date('Field_5','Field_6',df)
    subtrac_List = ['Field_1', 'Field_2', 'Field_43', 'Field_44', 'Field_7','Field_8', 'Field_9']
    subtract_2C = list(combinations(subtrac_List, 2))
    for l in subtract_2C:
        subtract_date(l[0],l[1],df)
      
    for cat in ['F', 'E', 'C', 'G', 'A']:
        subtract_date(f'{cat}_startDate', f'{cat}_endDate', df)
    for col in cat_cols:
        df[col] = df[col].astype("category")
    print(df.shape) 
    return df

def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(r"	", " ", s)
    s = re.sub(r" ", " ", s) # b'\xc2\xa0'
    return s

def str_normalize1(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s

def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)

    df["currentLocationLocationId"] = df["currentLocationLocationId"].apply(str_normalize).astype("category")
    df["homeTownLocationId"] = df["homeTownLocationId"].apply(str_normalize).astype("category")

    return df

def job_category(x):
    if type(x) == str:
        if "công nhân" in x or "cnv" in x or "cn" in x or "may công nghiệp" in x or "lao động" in x\
        or "thợ" in x or "coõng nhaõn trửùc tieỏp maựy may coõng nghieọp" in x or "c.n" in x or "lđ" in x:
            return "CN"
        elif "giáo viên" in x or "gv" in x or "gíao viên" in x:
            return "GV"
        elif "nhân viên" in x or "kế toán" in x or "cán bộ" in x or "nv" in x or "cb" in x or "nhõn viờn" in x:
            return "NV"
        elif "tài xế" in x or "lái" in x or "tài xê" in x:
            return "TX"
        elif "quản lý" in x or "phó phòng" in x or "hiệu phó" in x:
            return "QL"
        elif "undefined" in x:
            return "missing"
        elif "giám đốc" in x or "hiệu trưởng" in x:
            return "GĐ"
        elif "phục vụ" in x:
            return "PV"
        elif "chuyên viên" in x:
            return  "CV"
        elif "bác sĩ" in x or "dược sĩ" in x or "y sĩ" in x or "y sỹ" in x:
            return "BS"
        elif "y tá" in x:
            return "YT"
        elif "hộ sinh" in x:
            return "HS"
        elif "chủ tịch" in x:
            return "CT"
        elif "bếp" in x:
            return "ĐB"
        elif "sư" in x:
            return "KS"
        elif "dưỡng" in x:
            return "ĐD"
        elif "kỹ thuật" in x or "kĩ thuật" in x:
            return "KTV"
        elif "diễn viên" in x:
            return "DV"
        else:
            return "missing"
    else:
        return x 

def process_diaChi_maCv(df):
    df["maCv"] = df["maCv"].apply(str_normalize).apply(job_category).astype("category")
    return df

def combine_gender(s):
    x, y = s
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1).astype("category")
    return df

def process_ordinal(df):
    print(df.columns)

    df["subscriberCount"].replace(0, np.nan, inplace=True)
    df["friendCount"].replace(0, np.nan, inplace=True)

    df["Field_13"] = df["Field_13"].apply(lambda x: 1 if x == x else 0)
    df["Field_38"] = df["Field_38"].map({0: 0.0, 1: 1.0, "DN": np.nan, "TN": np.nan, "GD": np.nan})
    df["Field_62"] = df["Field_62"].map({"I": 1.0, "II": 2.0, "III": 3.0, "IV": 4.0, "V": 5.0, "Ngoài quốc doanh Quận 7": np.nan})
    df["Field_47"] = df["Field_47"].map({"Zezo": 0.0, "One": 1.0, "Two": 2.0, "Three": 3.0, "Four": 4.0})
    df["Field_66"] = df["Field_66"].map({"B": 0.0, "C": 1.0, "D": 2.0, "E": 3.0, "F": 4.0, "G": 5.0, "H": 6.0, "I": 7.0})

    df["Field_27"] = df["Field_27"].replace({0.0: np.nan})
    df["Field_28"] = df["Field_28"].replace({0.0: np.nan})
        
    for col in df.columns:
        if df[col].dtype.name == "object":
            df[col] = df[col].apply(str_normalize).astype("category")
            
    return df

def transform(df):
    df["gioiTinh"] = df["gioiTinh"].str.lower()
    df["maCv"] = df["maCv"].str.lower()
    #df = process_datetime_cols(df)
    df = process_gender(df)
    df = process_location(df)
    df = process_diaChi_maCv(df)
    df = process_ordinal(df)
    #df[unidecode_columns] = df[unidecode_columns].applymap(lambda x: unidecode(x).lower() if x == x else x)
    #print(df[unidecode_columns])
    #df[cat_features_count_encode] = df[cat_features_count_encode].astype('category')
    df["null_sum"] = df.isnull().sum(axis=1)
    DROP1 = DROP + DATE + DATETIME
    return df.drop(DROP1, 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from feature_engine import categorical_encoders as ce
print("\n@Data Pre-Processing")

print("\n+ Feature Engineering")

df_all_fe = transform(df_all)

df_all_fe.ngaySinh.fillna(0, inplace=True)

df_all_fe['Age'] = df_all_fe.ngaySinh.apply(lambda x: (2020 - int(x/10000)) if (x != 0) else x)
df_all_fe['Age'] = df_all_fe.Age.apply(lambda x: x if (x >= 18 and x <= 65) else 0)
df_all_fe['Field_34'] = df_all_fe.Field_34.apply(lambda x: str(x)[:6])
df_all_fe = df_all_fe.drop('ngaySinh', axis = 1)
#print(df_all_fe)
cols_select = [x for x in df_all_fe.columns if x not in DROP]
df_fe = df_all_fe[cols_select]
df_fe.replace([np.inf, -np.inf], -99999, inplace=True)
#print(df_fe)

y_label = train_df["label"]
train_fe = df_fe[df_fe["id"] < 53030]
test_fe = df_fe[df_fe["id"] >= 53030]

# Label-Encoding
lbl = LabelEncoder()
for col in df_fe.columns:
  if df_fe[col].dtype.name == "category":
    train_fe[col] = train_fe[col].astype(str)
    test_fe[col] = test_fe[col].astype(str)
    encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
    train_fe[col] = train_fe[col].fillna('None')
    test_fe[col] = test_fe[col].fillna('None')

    # Only take the common values in Train/Test set
    common_vals = list(set(train_fe[col]).intersection(set(test_fe[col])))

    # Take if vals appeared both 5 times
    common_vals = set(train_fe[col].value_counts()[train_fe[col].value_counts() > 4].index).intersection(test_fe[col].value_counts()[test_fe[col].value_counts()>4].index)

    # Replace not-common values with "Missing" or np.nan
    train_fe.loc[~train_fe[col].isin(common_vals), col] = 'Missing'
    test_fe.loc[~test_fe[col].isin(common_vals), col] = 'Missing'

    # Implement LE
    lbl.fit(train_fe[col].tolist() + test_fe[col].tolist())
    train_fe[col] = lbl.transform(train_fe[col])
    test_fe[col] = lbl.transform(test_fe[col])

print("train_df.shape = ", train_fe.shape, " | test_df.shape = ", test_fe.shape)


@Data Pre-Processing

+ Feature Engineering
Index(['id', 'Field_1', 'Field_2', 'Field_3', 'Field_4', 'Field_5', 'Field_6',
       'Field_7', 'Field_8', 'Field_9',
       ...
       'partner5_L', 'brief', 'num_of_phone', 'Field_78', 'Field_79',
       'Field_80', 'Field_81', 'Field_82', 'gender', 'null_sum'],
      dtype='object', length=196)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

train_df.shape =  (53030, 72)  | test_df.shape =  (20381, 72)


In [ ]:
temp_train = train_df[['label']]
#temp_test = test_df[['Field_2', 'Field_5', 'Field_6', 'Field_7', 'Field_8']]

In [ ]:
#Fill NaN values
train_fe = pd.concat([train_fe, temp_train], axis = 1)
train_fe.fillna(-1, inplace=True)
test_fe.fillna(-1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
# WOE Binning
bin_num_limit = 8
stop_limit = 0.05
count_distr_limit = 0.05
cd1 = '/content/drive/My Drive/kalapa'

##From here
woe_cols = [
    'Field_18', 'Field_20', 'Field_23',
    'Field_34', 'Field_45', 'Field_46', 'Field_48', 'Field_49', 'Field_55', 'Field_56',
    'Field_67', 'Field_71', 'Field_72', 'Field_73', 'Field_74', 'Field_75', 'Field_78', 'Field_79', 
    'Field_80', 'Field_81', 'Field_82', 'summary_1w', 'summary_1m', 'summary_3m', 'summary_6m',
    'currentLocationCity', 'homeTownCity', 'Age', 'A_numOrg', 'numOrg',
    'friendCount', 'gender', 'num_of_phone', 'subscriberCount', 'null_sum', 'topFriends'
]
##To there

commands = []
for col_name in woe_cols:
    os.makedirs(os.path.join(cd1, "woe/%s"%col_name), exist_ok=True)
    df1 = train_fe[["id", "label", col_name]]
    df2 = test_fe[["id", col_name]]
    df1.to_csv(os.path.join(cd1, "woe/%s/train.csv"%col_name), index=False, encoding="utf-8")
    df2.to_csv(os.path.join(cd1, "woe/%s/test.csv"%col_name), index=False, encoding="utf-8")

    print(f"Binning: {col_name}", f"bin_num_limit={bin_num_limit}", f"stop_limit={stop_limit}", f"count_distr_limit={count_distr_limit}")
    commands.append(
        f"Rscript woe.r {cd1}/woe {col_name} {bin_num_limit} {stop_limit} {count_distr_limit}"
    )

#procs = [Popen(c.strip().split()) for c in commands]
#for p in procs:
#    p.wait()


#START FROM HERE
train_woe_df = {}
test_woe_df = {}
for col_name in woe_cols:
    df1 = pd.read_csv(os.path.join(cd1, 'woe/%s/train_woe.csv'%col_name))
    df2 = pd.read_csv(os.path.join(cd1, 'woe/%s/test_woe.csv'%col_name))
    for c in df1.columns:
        train_woe_df[c] = df1[c]
    for c in df2.columns:
        test_woe_df[c] = df2[c]

train_fe_df = pd.DataFrame.from_dict(train_woe_df)
test_fe_df = pd.DataFrame.from_dict(test_woe_df)

print(" "*4, "After dropping: ", train_fe_df.shape, test_fe_df.shape)
print("-"*40)

print("Done!")
print("-"*60)

train_fe_df.to_csv(os.path.join(cd1, "train_fe.csv"), index=False, encoding="utf-8")
test_fe_df.to_csv(os.path.join(cd1, "test_fe.csv"), index=False, encoding="utf-8")


Binning: Field_18 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_20 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_23 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_34 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_45 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_46 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_48 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_49 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_55 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_56 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_67 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_71 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_72 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: Field_73 bin_num_limit=8 stop_limit=0.05 c

FileNotFoundError: ignored

In [ ]:
# 4. Data Spliting
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

print("\n@Learning")
n_folds = 5
seed = 2020
    
print("+ Data Splitting")

train_fe_df.label.replace("Good", 0, inplace=True)
train_fe_df.label.replace("Bad", 1, inplace=True)
print(f"Stratified {n_folds}-fold, seed={seed}")
y = train_fe_df["label"].values
#cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed) ##Update
cv = StratifiedShufleSplit(n_splits = n_folds, test_size = 0.2) ##Updated
for i, (train, val) in enumerate(cv.split(np.zeros(len(y)), y)):
    print("FOLD %d" % (i + 1))
    os.makedirs(os.path.join(cd1, "fold%d" % i), exist_ok=True)
    train_df, val_df = train_fe_df.loc[train], train_fe_df.loc[val]
    # use all positive examples for training and evaluation
    train_df = pd.concat([train_df, val_df[val_df.label == 1]])
    val_df = pd.concat([val_df, train_df[train_df.label == 1]])
    train_df.to_csv(os.path.join(cd1, "fold%d/train.csv" % i), index=False)
    val_df.to_csv(os.path.join(cd1, "fold%d/val.csv" % i), index=False)

print("-"*50)

In [ ]:
seed = 2020
n_trees = 767
max_depth = 17
min_samples_split = 2
min_samples_leaf = 1 
max_features = 'auto'
class_weight = None
bootstrap = True
n_folds = 5

#embeddings = pd.read_pickle("./data/embeddings.pkl").to_numpy(dtype=np.float32)

# submission input
X_submit = pd.read_csv(os.path.join(INPUT_DIR, "test_fe.csv"))
submit_id = X_submit.id.to_numpy(int)
submit_dict = {"id": submit_id}
X_submit.drop(columns=["id"], inplace=True)
X_submit = X_submit.to_numpy(dtype=np.float32)
#X_submit = np.concatenate([X_submit, embeddings[submit_id]], axis=1)
print(X_submit.shape)

# training and evaluation
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fold_aucs = []

fig, ax = plt.subplots()
for i in range(n_folds):
    print("FOLD %d" % (i + 1))

    train_df = pd.read_csv(os.path.join(cd1, "fold%d/train.csv" % i))
    val_df = pd.read_csv(os.path.join(cd1, "fold%d/val.csv" % i))
    train_id = train_df.id.to_numpy(int)
    val_id = val_df.id.to_numpy(int)
    train_df.drop(columns=["id"], inplace=True)
    val_df.drop(columns=["id"], inplace=True)
    
    y_train = train_df["label"].to_numpy(dtype=np.float32)
    X_train = train_df.drop(columns=["label"]).to_numpy(dtype=np.float32)
    #X_train = np.concatenate([X_train, embeddings[train_id]], axis=1)
    y_val = val_df["label"].to_numpy(dtype=np.float32)
    X_val = val_df.drop(columns=["label"]).to_numpy(dtype=np.float32)
    #X_val = np.concatenate([X_val, embeddings[val_id]], axis=1)
    print(X_train.shape, X_val.shape)

  
    clf = RandomForestClassifier(
        n_estimators=n_trees,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=seed,
        class_weight=class_weight,
        bootstrap=True,
        n_jobs=1
    )

    clf.fit(X_train, y_train)

    auc_ = roc_auc_score(y_val, clf.predict_proba(X_val)[:, 1])
    fold_aucs.append(auc_)
    print(f"val AUC = {auc_:.4f}")

    y_submit = clf.predict_proba(X_submit)[:, 1]
    submit_dict["fold%d" % i] = y_submit

    viz = plot_roc_curve(
        clf, X_val, y_val, name=f"ROC Fold {i}", alpha=0.3, lw=1, ax=ax
    )
    interp_tpr = interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Random", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="ROC Curves")
ax.legend(loc="lower right")
plt.savefig("roc.png")

print("Mean AUC = %0.4f, GINI %0.4f" % (mean_auc, 2 * mean_auc - 1.0))

print("\t".join(f"{x:.4f}" for x in fold_aucs))

##Updated
#Plot important feature
feature_imp = pd.Series(clf.feature_importances_,index=train_fe.columns).sort_values(ascending=False)
%matplotlib inline
# Creating a bar plot
plt.figure(figsize = (50, 50))
sns.barplot(x=feature_imp.head(100), y=feature_imp.head(100).index)

# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
#plt.legend()
plt.show()
##End

# averaging for submission
res_df = pd.DataFrame(submit_dict)
res_df["label"] = res_df[["fold%d" % i for i in range(n_folds)]].mean(axis=1)
res_df[["id", "label"]].to_csv("submission.csv", index=False)

for i in range(n_folds):
    res_df["label"] = res_df[f"fold{i}"]
    res_df[["id", "label"]].to_csv("submission_fold%d.csv" % i, index=False)


In [ ]:
for i in range (0, 5):
  print(i)

0
1
2
3
4


In [ ]:
print("\n@Rules")

for i in range(0, 5):
    sub_df = pd.read_csv("submission_fold%d.csv" % i)

    # Smoothing
    y = sub_df.label.to_numpy()
    rank = np.argsort(y)
    y_smooth = np.arange(len(rank)) * (1.0 / (len(rank) - 1))
    y[rank] = y_smooth
    sub_df.label = y

    rule_df = pd.read_csv("rules.csv", dtype=str, encoding="utf-8")
    test_df = pd.read_csv(os.path.join(cd1, "test.csv"), dtype=str, encoding="utf-8")

    mask = np.ones(sub_df.shape[0])

    for col in rule_df.columns:
        patterns = set(str_normalize_final(v) for v in rule_df[col].unique())
        patterns -= set(['nan'])
        if len(patterns) == 0:
            continue
    
        col_mask = test_df[col].apply(lambda x: 0. if str_normalize_final(x) in patterns else 1.)    
        mask *= col_mask.to_numpy()
    
    # Smoothing
    y = sub_df.label.to_numpy()
    org_idx = np.argwhere(mask).ravel()
    y_masked = y[org_idx]
    rank = np.argsort(y_masked)
    y_smooth = np.arange(1, len(rank) + 1) * (1.0 / (len(rank) + 1))
    y_masked[rank] = y_smooth
    y[org_idx] = y_masked

    sub_df.label = y * mask
    sub_df.to_csv("final_submission%d.csv" % i, index=False)